## **Connection to LLM
Core**

In [ ]:
import os

url_ollama_host = "INSERT NGROK URL HERE"

os.environ["OLLAMA_HOST"] = url_ollama_host

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sed 's#https://ollama.com/download#https://github.com/jmorganca/ollama/releases/download/v0.1.27#' | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama pull granite3-dense:8b

## **Installation of packages and choice of embeddings**

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.3 MB/s eta 0:00:00


In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="granite3-dense:8b", base_url=url_ollama_host, temperature=0.0)

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="mixedbread-ai/mxbai-embed-large-v1"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## **Directory Loader** (to load OpenAPI services specification)

In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
from langchain_community.document_loaders.directory import DirectoryLoader, TextLoader

In [ ]:
loader = DirectoryLoader("/content/input_files", show_progress=True, loader_cls=TextLoader)

In [ ]:
docs = loader.load()

100%|██████████| 14/14 [00:00<00:00, 2176.52it/s]


## **RAGAS evalution framework**

In [ ]:
!pip install ragas

Generation of the synthetic dataset

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.from_langchain(
    llm,
    llm,
    embeddings
)

In [ ]:
distributions = {
    simple: 0.5,
    multi_context: 0.25,
    reasoning: 0.25
}

In [ ]:
testset = generator.generate_with_langchain_docs(docs, 10, distributions, raise_exceptions= False)
test_df = testset.to_pandas()

embedding nodes:   0%|          | 0/44 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

ERROR:ragas.executor:Runner in Executor raised an exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/ragas/executor.py", line 79, in _aresults
    r = await future
  File "/usr/lib/python3.10/asyncio/tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "/usr/local/lib/python3.10/dist-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
  File "/usr/local/lib/python3.10/dist-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ragas/testset/evolutions.py", line 144, in evolve
    return await self.generate_datarow(
  File "/usr/local/lib/python3.10/dist-packages/ragas/testset/evolutions.py", line 236, in generate_datarow
    return DataRow(
  File "/usr/local/lib/python3.10/dist-packages/pydantic/v1/main.py", line 341, in __init__
    raise validation_error
pydantic.v1.error

In [ ]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the subset of attributes to include ...,"[{\n ""openapi"": ""3.1.0"",\n ""info"": {\n ""t...",nan,simple,[{'source': '/content/input_files/json_data_re...,True
1,What is the function of this API service and ...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The 'Pandas DataFrame DateTime Conversion Serv...,simple,[{'source': '/content/input_files/datetime_con...,True
2,What is the role of the given Factor Analysis...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The Factor Analysis feature selection service ...,simple,[{'source': '/content/input_files/factor_analy...,True
3,What kind of data structure is required as an...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",A Pandas DataFrame is required as an input for...,simple,[{'source': '/content/input_files/hierarchical...,True
4,What are the names of the independent variabl...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The independent variable names are specified i...,simple,[{'source': '/content/input_files/ridge_algori...,True
5,Which service handles removing columns from a ...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...","The first service, named 'Pandas DataFrame Col...",multi_context,[{'source': '/content/input_files/column_selec...,True
6,"When handling a Pandas DataFrame, under what c...","[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",-1,multi_context,[{'source': '/content/input_files/datetime_con...,True
7,What DataFrame would result from applying Fact...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The service applies Factor Analysis feature se...,reasoning,[{'source': '/content/input_files/factor_analy...,True
8,What's the preferred format for a Pandas DataF...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The Pandas DataFrame should be provided in JSO...,reasoning,[{'source': '/content/input_files/k-fold_cross...,True


In [ ]:
answers = []
for i in range(0, 9):
  question = test_df.iloc[i, 0]
  output = llm.invoke(question)
  answers.append(output)

In [ ]:
print(answers)

[" To determine the subset of attributes to include in the result when retrieving CSV data using an API, you'll need to refer to the documentation provided by the API provider. The available attributes and their corresponding column names in the CSV output may vary depending on the specific API.\n\nHere are some general steps you can follow to identify the desired attributes:\n\n1. Review the API documentation: Carefully read through the API's documentation, specifically looking for details about data retrieval, response formats (such as CSV), and available attributes or fields. Make a list of the attribute names that may be relevant to your use case.\n\n2. Test the API: Make API calls using different combinations of attribute names in the query parameters and inspect the resulting CSV files to see which columns are actually included in the output.\n\n3. Consult the API support team: If you're unable to identify the correct attribute names from the documentation or through testing, rea

In [ ]:
test_df['answer'] = answers

In [ ]:
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer
0,What are the subset of attributes to include ...,"[{\n ""openapi"": ""3.1.0"",\n ""info"": {\n ""t...",nan,simple,[{'source': '/content/input_files/json_data_re...,True,To determine the subset of attributes to incl...
1,What is the function of this API service and ...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The 'Pandas DataFrame DateTime Conversion Serv...,simple,[{'source': '/content/input_files/datetime_con...,True,I cannot directly answer that question as you...
2,What is the role of the given Factor Analysis...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The Factor Analysis feature selection service ...,simple,[{'source': '/content/input_files/factor_analy...,True,Factor Analysis is a statistical method used ...
3,What kind of data structure is required as an...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",A Pandas DataFrame is required as an input for...,simple,[{'source': '/content/input_files/hierarchical...,True,For Hierarchical Clustering feature selection...
4,What are the names of the independent variabl...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The independent variable names are specified i...,simple,[{'source': '/content/input_files/ridge_algori...,True,"In a Ridge Regression model, the independent ..."


Evaluation of metrics

In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, answer_similarity,  answer_relevancy, context_recall, context_precision
from datasets import Dataset

dataset = Dataset.from_pandas(test_df)

score = evaluate(dataset,metrics=[answer_relevancy, faithfulness, context_recall, context_precision], llm=llm, embeddings=embeddings)
score.to_pandas()

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer,answer_relevancy,faithfulness,context_recall,context_precision
0,What are the subset of attributes to include ...,"[{\n ""openapi"": ""3.1.0"",\n ""info"": {\n ""t...",nan,simple,[{'source': '/content/input_files/json_data_re...,True,To determine the subset of attributes to incl...,0.803970,1.000000,1.0,0.0
1,What is the function of this API service and ...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The 'Pandas DataFrame DateTime Conversion Serv...,simple,[{'source': '/content/input_files/datetime_con...,True,I cannot directly answer that question as you...,NaN,1.000000,NaN,1.0
2,What is the role of the given Factor Analysis...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The Factor Analysis feature selection service ...,simple,[{'source': '/content/input_files/factor_analy...,True,Factor Analysis is a statistical method used ...,0.958942,1.000000,1.0,1.0
3,What kind of data structure is required as an...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",A Pandas DataFrame is required as an input for...,simple,[{'source': '/content/input_files/hierarchical...,True,For Hierarchical Clustering feature selection...,0.914075,1.000000,1.0,1.0
4,What are the names of the independent variabl...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The independent variable names are specified i...,simple,[{'source': '/content/input_files/ridge_algori...,True,"In a Ridge Regression model, the independent ...",0.914585,0.666667,1.0,1.0
5,Which service handles removing columns from a ...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...","The first service, named 'Pandas DataFrame Col...",multi_context,[{'source': '/content/input_files/column_selec...,True,To remove columns from a given DataFrame obje...,0.813716,1.000000,NaN,1.0
6,"When handling a Pandas DataFrame, under what c...","[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",-1,multi_context,[{'source': '/content/input_files/datetime_con...,True,Columns in a Pandas DataFrame should be conve...,0.912333,0.800000,0.0,0.0
7,What DataFrame would result from applying Fact...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The service applies Factor Analysis feature se...,reasoning,[{'source': '/content/input_files/factor_analy...,True,Applying Factor Analysis on selected columns ...,0.787750,1.000000,1.0,1.0
8,What's the preferred format for a Pandas DataF...,"[{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""t...",The Pandas DataFrame should be provided in JSO...,reasoning,[{'source': '/content/input_files/k-fold_cross...,True,"In k-fold cross-validation, each observation ...",0.949942,0.833333,1.0,1.0
